In [1]:
import pandas as pd
import requests
from io import StringIO

from nst_scraper import nst_on_ice_scraper

pd.set_option('display.max_columns', None)


In [2]:
player_df = nst_on_ice_scraper(fromseason=20232024, thruseason=20232024, startdate='', enddate='')
player_df


,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,ipp,shots,sh%,ixg,icf,iff,iscf,ihdcf,rush_attempts,rebounds_created,pim,total_penalties,minor,major,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
0,Ryan Suter,DAL,D,82,1403.833333,2,14,6,8,16,26.23,99,2.02,3.72,245,159,67,6,2,15,24,12,12,0,0,0,28,14,75,92,86,0,0,-
1,Jeff Carter,PIT,C,72,664.900000,5,3,1,2,8,61.54,65,7.69,5.86,122,96,61,31,6,13,10,5,5,0,0,3,16,16,75,32,25,169,87,66.02
2,Zach Parise,COL,L,30,340.616667,4,3,1,2,7,70.00,43,9.30,4.56,68,54,41,23,4,11,6,3,3,0,0,2,4,10,20,22,23,11,21,34.38
3,Brent Burns,CAR,D,82,1276.683333,5,12,5,7,17,26.56,134,3.73,4.75,417,225,96,10,6,43,18,9,9,0,0,5,43,30,34,33,72,0,0,-
4,Corey Perry,"CHI, EDM",R,54,600.266667,9,7,4,3,16,53.33,61,14.75,7.57,114,92,71,37,4,13,46,17,13,4,0,14,17,15,42,66,17,0,1,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,Jiri Smejkal,OTT,L,20,164.316667,1,1,1,0,2,100.00,8,12.50,1.82,27,21,13,7,0,3,2,1,1,0,0,2,5,6,32,14,13,9,15,37.50
920,Nikolas Matinpalo,OTT,D,4,27.333333,0,0,0,0,0,0.00,2,0.00,0.02,7,3,2,0,0,0,0,0,0,0,0,0,1,0,4,7,2,0,0,-
921,Waltteri Merela,T.B,C,19,170.500000,1,0,0,0,1,25.00,17,5.88,1.36,27,18,19,10,2,2,4,2,2,0,0,4,2,1,30,32,6,1,3,25.00
922,Patrik Koch,ARI,D,1,9.333333,0,0,0,0,0,0.00,1,0.00,0.01,4,1,0,0,1,0,10,1,0,0,1,0,0,0,4,2,1,0,0,-


In [3]:
goalie_df = nst_on_ice_scraper(fromseason=20232024, thruseason=20232024, startdate='', enddate='', pos='g')
goalie_df


,player,team,gp,toi,shots_against,saves,goals_against,sv%,gaa,gsaa,xg_against,hd_shots_against,hd_saves,hd_goals_against,hdsv%,hdgaa,hdgsaa,md_shots_against,md_saves,md_goals_against,mdsv%,mdgaa,mdgsaa,ld_shots_against,ld_saves,ld_goals_against,ldsv%,ldgaa,ldgsaa,rush_attempts_against,rebound_attempts_against,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,40,1758.750000,802,738,64,0.920,2.18,5.40,65.04,170,142,28,0.835,0.96,2.91,196,172,24,0.878,0.82,-2.53,394,382,12,0.970,0.41,0.07,49,125,37.33,21.02
1,Jonathan Quick,NYR,27,1283.333333,618,560,58,0.906,2.71,-4.52,58.21,164,134,30,0.817,1.40,-0.19,170,149,21,0.876,0.98,-2.38,254,247,7,0.972,0.33,0.78,40,124,33.47,18.93
2,James Reimer,DET,25,1094.083333,551,496,55,0.900,3.02,-7.32,47.93,130,100,30,0.769,1.65,-6.37,140,122,18,0.871,0.99,-2.67,253,246,7,0.972,0.38,0.75,37,89,36.94,19.56
3,Semyon Varlamov,NYI,28,1301.083333,646,599,47,0.927,2.17,8.90,58.18,161,137,24,0.851,1.11,5.27,150,137,13,0.913,0.60,3.43,302,292,10,0.967,0.46,-0.75,46,143,34.86,22.83
4,Jacob Markstrom,CGY,48,2268.283333,1063,973,90,0.915,2.38,1.99,101.53,302,262,40,0.868,1.06,14.90,263,230,33,0.875,0.87,-4.19,449,432,17,0.962,0.45,-3.24,78,203,33.56,22.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Arvid Soderblom,CHI,32,1429.050000,768,681,87,0.887,3.65,-20.54,70.91,210,162,48,0.771,2.02,-9.82,199,177,22,0.889,0.92,-0.20,325,308,17,0.948,0.71,-7.04,49,146,33.31,18.53
94,Jet Greaves,CBJ,9,442.250000,254,232,22,0.913,2.98,-0.02,21.60,59,45,14,0.763,1.90,-3.27,59,55,4,0.932,0.54,2.46,122,118,4,0.967,0.54,-0.26,11,42,36.22,20.05
95,Matt Murray,DAL,1,47.050000,16,16,0,1.000,0.00,1.38,0.89,1,1,0,1.000,0.00,0.18,2,2,0,1.000,0.00,0.22,11,11,0,1.000,0.00,0.34,2,2,46.06,-
96,Yaniv Perets,CAR,1,10.666667,1,1,0,1.000,0.00,0.09,0.04,0,0,0,-,0.00,0.00,1,1,0,1.000,0.00,0.11,0,0,0,-,0.00,0.00,0,0,28.00,-


In [4]:
class Lineup:
    def __init__(self, name):
        self.name = name
        self.forwards = [None] * 12  # 12 Forward Slots
        self.defense = [None] * 6    # 6 Defense Slots
        self.goalie = None            # 1 Goalie Slot

    def add_forward(self, player, slot):
        if 0 <= slot < 12:
            self.forwards[slot] = player
        else:
            raise IndexError("Forward slot must be between 0 and 11.")

    def add_defense(self, player, slot):
        if 0 <= slot < 6:
            self.defense[slot] = player
        else:
            raise IndexError("Defense slot must be between 0 and 5.")

    def set_goalie(self, player):
        self.goalie = player

    def display_lineup(self):
        print(f"Lineup: {self.name}")
        print("Forwards:")
        for idx, player in enumerate(self.forwards):
            print(f"  Slot {idx + 1}: {player if player else 'Empty'}")
        print("Defense:")
        for idx, player in enumerate(self.defense):
            print(f"  Slot {idx + 1}: {player if player else 'Empty'}")
        print(f"Goalie: {self.goalie if self.goalie else 'Empty'}\n")


In [5]:
# Creating two lineup objects
lineup1 = Lineup("Lineup 1")
lineup2 = Lineup("Lineup 2")

# Example of loading players from DataFrames
# Assuming player_df and goalie_df are already defined

# Adding forwards to lineup1
for i in range(12):
    if i < len(player_df):
        lineup1.add_forward(player_df.iloc[i]['player'], i)

# Adding defense to lineup1
for i in range(6):
    if i < len(player_df):
        lineup1.add_defense(player_df.iloc[i + 12]['player'], i)

# Setting goalie for lineup1
if not goalie_df.empty:
    lineup1.set_goalie(goalie_df.iloc[0]['player'])

# Repeat similar steps for lineup2 as needed

# Display the lineup
lineup1.display_lineup()

Lineup: Lineup 1
Forwards:
  Slot 1: Ryan Suter
  Slot 2: Jeff Carter
  Slot 3: Zach Parise
  Slot 4: Brent Burns
  Slot 5: Corey Perry
  Slot 6: Joe Pavelski
  Slot 7: Mark Giordano
  Slot 8: Alex Ovechkin
  Slot 9: Evgeni Malkin
  Slot 10: Blake Wheeler
  Slot 11: Alex Goligoski
  Slot 12: Sidney Crosby
Defense:
  Slot 1: Jack Johnson
  Slot 2: Anze Kopitar
  Slot 3: Marc Staal
  Slot 4: T.J. Oshie
  Slot 5: Andrew Cogliano
  Slot 6: Marc-Edouard Vlasic
Goalie: Marc-Andre Fleury



In [6]:
from team_utils import get_team_roster

get_team_roster('TOR', 20232024)


{'forwards': [{'id': 8481720,
   'headshot': 'https://assets.nhle.com/mugs/nhl/20232024/TOR/8481720.png',
   'firstName': {'default': 'Nick',
    'cs': 'Nicholas',
    'de': 'Nicholas',
    'es': 'Nicholas',
    'fi': 'Nicholas',
    'sk': 'Nicholas',
    'sv': 'Nicholas'},
   'lastName': {'default': 'Abruzzese'},
   'sweaterNumber': 26,
   'positionCode': 'C',
   'shootsCatches': 'L',
   'heightInInches': 71,
   'weightInPounds': 180,
   'heightInCentimeters': 180,
   'weightInKilograms': 82,
   'birthDate': '1999-06-04',
   'birthCity': {'default': 'Slate Hill'},
   'birthCountry': 'USA',
   'birthStateProvince': {'default': 'NY'}},
  {'id': 8480980,
   'headshot': 'https://assets.nhle.com/mugs/nhl/20232024/TOR/8480980.png',
   'firstName': {'default': 'Connor'},
   'lastName': {'default': 'Dewar'},
   'sweaterNumber': 24,
   'positionCode': 'C',
   'shootsCatches': 'L',
   'heightInInches': 70,
   'weightInPounds': 183,
   'heightInCentimeters': 178,
   'weightInKilograms': 83,
   '

In [7]:
from pbp_utils import get_matchup_games

games = get_matchup_games('2024-11-11', '2024-11-17')
pd.DataFrame({
    'game_id': games['game_ids']['id'],
    'date': games['game_ids']['date']
})



,game_id,date
0,2024020239,2024-11-11
1,2024020240,2024-11-11
2,2024020241,2024-11-11
3,2024020242,2024-11-11
4,2024020243,2024-11-11
5,2024020244,2024-11-11
6,2024020245,2024-11-12
7,2024020247,2024-11-12
8,2024020246,2024-11-12
9,2024020248,2024-11-12


In [8]:
from game_utils import get_game_boxscore, display_boxscore

temp_data = get_game_boxscore(2024020255)

In [9]:
away_skaters, away_goalies, home_skaters, home_goalies = display_boxscore(temp_data)

away_skaters


,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,team
0,8477960,9,A. Kempe,R,2,0,2,0,0,1,0,4,0.000000,19:11,1,23,0,0,Away
1,8471685,11,A. Kopitar,C,0,2,2,0,0,0,0,1,0.769231,19:29,0,23,0,0,Away
2,8479675,12,T. Moore,L,0,2,2,0,0,1,0,0,0.000000,18:52,1,24,0,0,Away
3,8482155,14,A. Laferriere,R,0,0,0,-3,0,2,0,1,0.000000,15:46,0,22,0,1,Away
4,8477942,22,K. Fiala,L,0,0,0,-1,2,2,0,1,0.000000,18:00,2,22,2,0,Away
5,8476479,24,P. Danault,C,0,0,0,-1,2,1,0,0,0.636364,18:11,1,25,1,1,Away
6,8480851,26,A. Thomas,C,0,0,0,0,0,1,0,0,0.000000,11:09,1,15,0,0,Away
7,8477998,37,W. Foegele,L,0,0,0,-1,0,2,0,1,0.000000,15:37,1,25,2,0,Away
8,8481732,47,A. Lee,L,0,0,0,-1,0,3,0,2,0.000000,10:43,2,13,0,1,Away
9,8482124,55,Q. Byfield,R,0,0,0,-1,0,1,0,1,0.500000,13:46,0,22,2,0,Away


In [13]:
away_goalies


,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,team
0,8479496,31,{'default': 'D. Rittich'},G,4/5,0/0,0/0,4/5,0.8,1,0,0,0,1,14:41,False,L,5,4,Away
1,8475311,35,{'default': 'D. Kuemper'},G,17/19,1/1,0/0,18/20,0.9,2,0,0,0,2,44:32,True,NaN,20,18,Away
